In [4]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))

import pandas as pd
from pycaret.classification import ClassificationExperiment
import src.utils as utils
import src.calcEMA as calc_utils
import src.myenv as myenv
from datetime import datetime

from itertools import combinations
import plotly.express as px

<built-in method __dir__ of sqlite3.Connection object at 0x7fd6b46f5440>


In [5]:
# Variables
# ETCUSDT BTCUSDT
# symbol = 'ETHUSDT'
symbol = 'ETHUSDT'
interval = '1h'
# lightgbm  xgboost lr et rf
estimator = 'knn'
_compare_models = False

start_train_date = '2010-01-01'  # train < and test >=
start_test_date = '2023-01-01'  # train < and test >=

stop_loss = 1.0
# regression_times = 0  # 24 * 30 * 2  # horas
times_regression_PnL = 24
normalize = True
use_gpu = False
tune_model = False
apply_combination_features = True

In [ ]:
cols = myenv.all_klines_cols.copy()
cols.remove('ignore')
data = utils.get_data(symbol=symbol, save_database=False, interval=interval, tail=-1, columns=cols, parse_dates=True, start_date=start_train_date)
data = data[data['open_time'] >= start_train_date]
data = utils.parse_type_fields(data, parse_dates=True)
data = utils.adjust_index(data)
data.info()
data

In [ ]:
data = calc_utils.calc_RSI(data)
data.info()
data

In [ ]:
data = calc_utils.calc_ema_periods(data, periods_of_time=[times_regression_PnL, 200])
data.info()
data

In [ ]:
data = utils.regression_PnL(
    data=data,
    label=myenv.label,
    diff_percent=float(stop_loss),
    max_regression_profit_and_loss=int(times_regression_PnL),
    drop_na=True,
    drop_calc_cols=True,
    strategy=None)
data.dropna(inplace=True)
data.info()

In [ ]:
data.dropna(inplace=True)
data.info()

In [ ]:
perc_data_label = data[[myenv.label, 'open_time']].groupby(myenv.label).count()
perc_data_label['perc'] = perc_data_label['open_time'] / len(data)
perc_data_label

In [ ]:
train_data = data[(data['open_time'] >= start_train_date) & (data['open_time'] < start_test_date)]
train_data = train_data.sort_values(myenv.date_features)
train_data

In [ ]:
test_data = data[data['open_time'] >= start_test_date]
test_data = test_data.sort_values(myenv.date_features)
test_data

In [ ]:
# BTCUSDT 1h best params: close,volume,quote_asset_volume,number_of_trades,rsi
# text_numeric_features = 'volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,rsi,ema_24p,ema_200p'.split(',')
#text_numeric_features = 'close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,rsi,ema_24p,ema_200p'
text_numeric_features = f'volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ema_{int(times_regression_PnL)}p'
#text_numeric_features2 = f'volume,taker_buy_base_asset_volume,ema_{int(times_regression_PnL)}p'

numeric_features = text_numeric_features.split(',')
print(f'Numeric Features: {numeric_features} - size: {len(numeric_features)}\n')

combination_numeric_features = []
if apply_combination_features:
	for size in range(1, len(numeric_features) + 1): 
		comb = map(list, combinations(numeric_features, size))	
		for c in comb:
			res = ''
			for j in c:
				res += f'{j},'
			combination_numeric_features.append(res[0:len(res) - 1])
else:
	combination_numeric_features = [text_numeric_features]

print(f'Combination Numeric Features: \n{combination_numeric_features}')

In [ ]:
from datetime import datetime
import os
from imblearn.under_sampling import InstanceHardnessThreshold, RepeatedEditedNearestNeighbours, AllKNN


#iht = InstanceHardnessThreshold(estimator=KNeighborsClassifier())

# fix_imbalance_method: condensednearestneighbour, editednearestneighborus, repeatededitednearestneighbours, allknn, instancehardnessthreshold, nearmiss, neighbourhoodcleaningrule, onesidedselection, randomundersampler, tomeklinks, randomoversampler, smote, smotenc, smoten, adasyn, borderlinesmote, kmeanssmote, svmsmote, smoteenn, smotetomek.
# 'condensednearestneighbour,editednearestneighborus,repeatededitednearestneighbours,allknn,instancehardnessthreshold,nearmiss,neighbourhoodcleaningrule,onesidedselection,randomundersampler,tomeklinks,randomoversampler,smote,smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
# 'smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
#imbalance_list = 'editednearestneighborus,repeatededitednearestneighbours,allknn,instancehardnessthreshold,nearmiss,neighbourhoodcleaningrule,onesidedselection,randomundersampler,tomeklinks,randomoversampler,smote,smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
imbalance_list = 'allknn,instancehardnessthreshold,repeatededitednearestneighbours'.split(',')
simulation_results_filename = f'./resultado_simulacao_{symbol}_{interval}.csv'

df_resultado_simulacao = pd.DataFrame()
has_simulation_file = os.path.exists(simulation_results_filename)
if has_simulation_file:
  df_resultado_simulacao = pd.read_csv(simulation_results_filename, sep=';')

for aux_numeric_features in combination_numeric_features:
  experiement = ClassificationExperiment()
  features = []
  features += ['open_time', myenv.label]
  features += aux_numeric_features.split(',')
  print(f'features: {features}')
  for fix_imbalance_method in imbalance_list:
    if has_simulation_file:
      chave = (df_resultado_simulacao['symbol'] == symbol) & \
          (df_resultado_simulacao['interval'] == interval) & \
          (df_resultado_simulacao['estimator'] == estimator) & \
          (df_resultado_simulacao['fix_imbalance_method'] == fix_imbalance_method) & \
          (df_resultado_simulacao['features'] == str(features))

      if chave.sum() > 0:
        print(f'fix_imbalance_method: {fix_imbalance_method} already exists')
        continue

    try:
      print(f'fix_imbalance_method: {fix_imbalance_method}')
      fim = fix_imbalance_method
      match(fix_imbalance_method):
        case 'repeatededitednearestneighbours':
          fim = RepeatedEditedNearestNeighbours(kind_sel='all', max_iter=100, n_jobs=20, n_neighbors=3, sampling_strategy='auto')
        case 'instancehardnessthreshold':
          fim = InstanceHardnessThreshold(cv=5, estimator=None, n_jobs=20, random_state=123, sampling_strategy='auto')
        case 'allknn':
          fim = AllKNN(allow_minority=False, kind_sel='all', n_jobs=20, n_neighbors=3, sampling_strategy='auto')

      setup = experiement.setup(
          data=train_data[features].copy(),
          train_size=myenv.train_size,
          target=myenv.label,
          numeric_features=aux_numeric_features.split(','),
          date_features=['open_time'],
          create_date_columns=["hour", "day", "month"],
          data_split_shuffle=False,
          data_split_stratify=False,
          fix_imbalance=True,
          fix_imbalance_method=fim,
          remove_outliers=True,
          fold_strategy='timeseries', 
          fold=3,
          session_id=123,
          normalize=normalize,
          use_gpu=False,
          verbose=True,
          n_jobs=20,
          log_experiment=False,
      )

      if _compare_models:
        best = setup.compare_models()
        estimator = setup.pull().index[0]
        print('Estimator: ' + estimator)
      else:
        best = setup.create_model(estimator, n_jobs=20)

      if tune_model:
        best = setup.tune_model(best)

      # predict on test set
      # holdout_pred = setup.predict_model(best)
      # print('Holdout Score:', holdout_pred['prediction_score'].mean())
      # print('Holdout Score Group:\n', holdout_pred[[myenv.label, 'prediction_score']].groupby(myenv.label).mean())

      predict = setup.predict_model(best, data=test_data.drop(columns=[myenv.label]))
      predict[myenv.label] = test_data[myenv.label]
      predict['_score'] = predict['prediction_label'] == predict[myenv.label]
      # print('Predict Score Mean:', predict['_score'].mean())
      # print('Predict Score Mean Group:\n', predict[[myenv.label, '_score']].groupby(myenv.label).mean())

      final_model = setup.finalize_model(best)

      ajusted_test_data = test_data.drop(myenv.label, axis=1)
      df_final_predict, res_score = utils.validate_score_test_data(
          setup,
          final_model,
          myenv.label,
          test_data,
          ajusted_test_data)

      # df_final_predict.info()
      # print('Final Score Mean:', res_score.mean().values[0])
      # print('Final Score Group:\n', res_score)

      start_test_date = df_final_predict['open_time'].min()
      end_test_date = df_final_predict['open_time'].max()

      # print('Simule Trading:')
      # print(f'Min Data: {start_test_date}')
      # print(f'Max Data: {end_test_date}')
      # saldo_inicial = 100.0
      # saldo_final = utils.simule_trading_crypto2(df_final_predict, start_test_date, end_test_date, saldo_inicial, stop_loss)
      # print(f'>>>> Saldo Final: {saldo_final} - features: {features}\n\n')

      result_simulado = {}
      result_simulado['date'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
      result_simulado['symbol'] = symbol
      result_simulado['interval'] = interval
      result_simulado['estimator'] = estimator
      result_simulado['fix_imbalance_method'] = fix_imbalance_method
      result_simulado['stop_loss'] = stop_loss
      result_simulado['times_regression_profit_and_loss'] = times_regression_PnL
      result_simulado['features'] = features
      # result_simulado['final_value'] = saldo_final

      if res_score is not None:
        for i in range(0, len(res_score.index.values)):
          field = res_score.index.values[i].split('_')[0]
          result_simulado[field] = round(res_score["_score"].values[i], 4)
        result_simulado['max_score'] = max(result_simulado['CAI'], result_simulado['SOBE'])

      df_resultado_simulacao = pd.concat([df_resultado_simulacao, pd.DataFrame([result_simulado])], ignore_index=True)
      df_resultado_simulacao.sort_values('max_score', inplace=True)

      df_resultado_simulacao.to_csv(simulation_results_filename, sep=';', index=False)
    except Exception as e:
      print(e)
      continue

In [26]:
pd.set_option('display.max_columns', None)

simulation_results_filename = f'{myenv.datadir}/resultado_simulacao_{symbol}_{interval}.csv'

res = pd.read_csv(simulation_results_filename, sep=';')
res.sort_values('CAI')

,data,symbol,interval,estimator,imbalance_method,stop_loss,regression_times,times_regression_profit_and_loss,CAI,ESTAVEL,SOBE,max_score,numeric_features,regression_features,train_size,use-all-data-to-train,start_train_date,start_test_date,fold,no-tune,model_name,arguments
7,2023-10-20 13:10:39,ETHUSDT,1h,knn,repeatededitednearestneighbours,1.0,0,24,0.0404,0.9124,0.0835,0.0835,"volume,number_of_trades,rsi,ema_24p",NaN,0.7,False,2017-11-01,2023-07-22 13:00:00,3,True,<< NOT SAVED >>,"['-batch-training', '-calc-rsi', '-normalize', '-no-tune', '-n-jobs=20', '-symbol-list=BTCUSDT,ETHUSDT,BNBUSDT,XRPUSDT,ADAUSDT,DOGEUSDT,SOLUSDT,TRXUSDT,DOTUSDT,MATICUSDT,LTCUSDT,BCHUSDT,AVAXUSDT,XLMUSDT,LINKUSDT,XMRUSDT,UNIUSDT,ATOMUSDT', '-interval-list=1h', '-estimator-list=knn', '-stop-loss-list=1.0', '-numeric-features=volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume', '-regression-PnL-list=24', '-regression-times-list=0', '-start-train-date=2017-11-01', '-start-test-date=2023-07-01']"
5,2023-10-20 13:39:27,ETHUSDT,1h,knn,repeatededitednearestneighbours,1.0,0,24,0.0472,0.9078,0.0817,0.0817,"volume,number_of_trades,rsi,ema_24p,ema_200p",NaN,0.7,False,2017-11-01,2023-07-22 13:00:00,3,True,<< NOT SAVED >>,"['-batch-training', '-calc-rsi', '-normalize', '-no-tune', '-n-jobs=20', '-symbol-list=BTCUSDT,ETHUSDT,BNBUSDT,XRPUSDT,ADAUSDT,DOGEUSDT,SOLUSDT,TRXUSDT,DOTUSDT,MATICUSDT,LTCUSDT,BCHUSDT,AVAXUSDT,XLMUSDT,LINKUSDT,XMRUSDT,UNIUSDT,ATOMUSDT', '-interval-list=1h', '-estimator-list=knn', '-stop-loss-list=1.0', '-numeric-features=volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume', '-regression-PnL-list=24', '-regression-times-list=0', '-start-train-date=2017-11-01', '-start-test-date=2023-07-01']"
25,2023-10-20 13:11:01,ETHUSDT,1h,knn,repeatededitednearestneighbours,1.0,0,24,0.0499,0.9124,0.0944,0.0944,"volume,number_of_trades,rsi,ema_200p",NaN,0.7,False,2017-11-01,2023-07-22 13:00:00,3,True,<< NOT SAVED >>,"['-batch-training', '-calc-rsi', '-normalize', '-no-tune', '-n-jobs=20', '-symbol-list=BTCUSDT,ETHUSDT,BNBUSDT,XRPUSDT,ADAUSDT,DOGEUSDT,SOLUSDT,TRXUSDT,DOTUSDT,MATICUSDT,LTCUSDT,BCHUSDT,AVAXUSDT,XLMUSDT,LINKUSDT,XMRUSDT,UNIUSDT,ATOMUSDT', '-interval-list=1h', '-estimator-list=knn', '-stop-loss-list=1.0', '-numeric-features=volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume', '-regression-PnL-list=24', '-regression-times-list=0', '-start-train-date=2017-11-01', '-start-test-date=2023-07-01']"
33,2023-10-20 12:46:12,ETHUSDT,1h,knn,repeatededitednearestneighbours,1.0,0,24,0.0593,0.8802,0.0998,0.0998,"volume,number_of_trades,ema_200p",NaN,0.7,False,2017-11-01,2023-07-22 13:00:00,3,True,<< NOT SAVED >>,"['-batch-training', '-calc-rsi', '-normalize', '-no-tune', '-n-jobs=20', '-symbol-list=BTCUSDT,ETHUSDT,BNBUSDT,XRPUSDT,ADAUSDT,DOGEUSDT,SOLUSDT,TRXUSDT,DOTUSDT,MATICUSDT,LTCUSDT,BCHUSDT,AVAXUSDT,XLMUSDT,LINKUSDT,XMRUSDT,UNIUSDT,ATOMUSDT', '-interval-list=1h', '-estimator-list=knn', '-stop-loss-list=1.0', '-numeric-features=volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume', '-regression-PnL-list=24', '-regression-times-list=0', '-start-train-date=2017-11-01', '-start-test-date=2023-07-01']"
51,2023-10-20 13:11:23,ETHUSDT,1h,knn,repeatededitednearestneighbours,1.0,0,24,0.0606,0.8975,0.1071,0.1071,"volume,number_of_trades,ema_24p,ema_200p",NaN,0.7,False,2017-11-01,2023-07-22 13:00:00,3,True,<< NOT SAVED >>,"['-batch-training', '-calc-rsi', '-normalize', '-no-tune', '-n-jobs=20', '-symbol-list=BTCUSDT,ETHUSDT,BNBUSDT,XRPUSDT,ADAUSDT,DOGEUSDT,SOLUSDT,TRXUSDT,DOTUSDT,MATICUSDT,LTCUSDT,BCHUSDT,AVAXUSDT,XLMUSDT,LINKUSDT,XMRUSDT,UNIUSDT,ATOMUSDT', '-interval-list=1h', '-estimator-list=knn', '-stop-loss-list=1.0', '-numeric-features=volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume', '-regression-PnL-list=24', '-regress

In [27]:
res.sort_values('SOBE')

,data,symbol,interval,estimator,imbalance_method,stop_loss,regression_times,times_regression_profit_and_loss,CAI,ESTAVEL,SOBE,max_score,numeric_features,regression_features,train_size,use-all-data-to-train,start_train_date,start_test_date,fold,no-tune,model_name,arguments
202,2023-10-20 13:26:04,ETHUSDT,1h,knn,repeatededitednearestneighbours,1.0,0,24,0.2399,0.8306,0.0254,0.2399,"number_of_trades,rsi,ema_24p,ema_200p",NaN,0.7,False,2017-11-01,2023-07-22 13:00:00,3,True,<< NOT SAVED >>,"['-batch-training', '-calc-rsi', '-normalize', '-no-tune', '-n-jobs=20', '-symbol-list=BTCUSDT,ETHUSDT,BNBUSDT,XRPUSDT,ADAUSDT,DOGEUSDT,SOLUSDT,TRXUSDT,DOTUSDT,MATICUSDT,LTCUSDT,BCHUSDT,AVAXUSDT,XLMUSDT,LINKUSDT,XMRUSDT,UNIUSDT,ATOMUSDT', '-interval-list=1h', '-estimator-list=knn', '-stop-loss-list=1.0', '-numeric-features=volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume', '-regression-PnL-list=24', '-regression-times-list=0', '-start-train-date=2017-11-01', '-start-test-date=2023-07-01']"
55,2023-10-20 13:07:14,ETHUSDT,1h,knn,repeatededitednearestneighbours,1.0,0,24,0.1092,0.9320,0.0290,0.1092,"volume,quote_asset_volume,rsi,ema_200p",NaN,0.7,False,2017-11-01,2023-07-22 13:00:00,3,True,<< NOT SAVED >>,"['-batch-training', '-calc-rsi', '-normalize', '-no-tune', '-n-jobs=20', '-symbol-list=BTCUSDT,ETHUSDT,BNBUSDT,XRPUSDT,ADAUSDT,DOGEUSDT,SOLUSDT,TRXUSDT,DOTUSDT,MATICUSDT,LTCUSDT,BCHUSDT,AVAXUSDT,XLMUSDT,LINKUSDT,XMRUSDT,UNIUSDT,ATOMUSDT', '-interval-list=1h', '-estimator-list=knn', '-stop-loss-list=1.0', '-numeric-features=volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume', '-regression-PnL-list=24', '-regression-times-list=0', '-start-train-date=2017-11-01', '-start-test-date=2023-07-01']"
59,2023-10-20 13:06:52,ETHUSDT,1h,knn,repeatededitednearestneighbours,1.0,0,24,0.1105,0.9343,0.0290,0.1105,"volume,quote_asset_volume,rsi,ema_24p",NaN,0.7,False,2017-11-01,2023-07-22 13:00:00,3,True,<< NOT SAVED >>,"['-batch-training', '-calc-rsi', '-normalize', '-no-tune', '-n-jobs=20', '-symbol-list=BTCUSDT,ETHUSDT,BNBUSDT,XRPUSDT,ADAUSDT,DOGEUSDT,SOLUSDT,TRXUSDT,DOTUSDT,MATICUSDT,LTCUSDT,BCHUSDT,AVAXUSDT,XLMUSDT,LINKUSDT,XMRUSDT,UNIUSDT,ATOMUSDT', '-interval-list=1h', '-estimator-list=knn', '-stop-loss-list=1.0', '-numeric-features=volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume', '-regression-PnL-list=24', '-regression-times-list=0', '-start-train-date=2017-11-01', '-start-test-date=2023-07-01']"
63,2023-10-20 14:01:56,ETHUSDT,1h,knn,repeatededitednearestneighbours,1.0,0,24,0.1132,0.9101,0.0327,0.1132,"number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,rsi,ema_24p,ema_200p",NaN,0.7,False,2017-11-01,2023-07-22 13:00:00,3,True,<< NOT SAVED >>,"['-batch-training', '-calc-rsi', '-normalize', '-no-tune', '-n-jobs=20', '-symbol-list=BTCUSDT,ETHUSDT,BNBUSDT,XRPUSDT,ADAUSDT,DOGEUSDT,SOLUSDT,TRXUSDT,DOTUSDT,MATICUSDT,LTCUSDT,BCHUSDT,AVAXUSDT,XLMUSDT,LINKUSDT,XMRUSDT,UNIUSDT,ATOMUSDT', '-interval-list=1h', '-estimator-list=knn', '-stop-loss-list=1.0', '-numeric-features=volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume', '-regression-PnL-list=24', '-regression-times-list=0', '-start-train-date=2017-11-01', '-start-test-date=2023-07-01']"
37,2023-10-20 13:58:48,ETHUSDT,1h,knn,repeatededitednearestneighbours,1.0,0,24,0.1024,0.8710,0.0327,0.1024,"volume,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,rsi,ema_24p,ema_200p",NaN,0.7,False,2017-11-01,2023-07-22 13:00:00,3,True,<< NOT SAVED >>,"['-batch-training', '-calc-rsi', '-normalize', '-no-tune', '-n-jobs=20', '-symbol-list=BTCUSDT,ETHUSDT,BNBUSDT,XRPUSDT,ADAUSDT,DOGEUSDT,SOLUSDT,TRXUSDT,DOTUSDT,MATICUSDT,LTCUSDT,BCHUSDT,AVAXUSDT,XLMUSDT,LINKUSDT,XMRUSDT,UNIUSDT,ATOMUSDT', '-interval-list=1h', '-estimator-list=knn', '-stop-loss-list=1.0', '-numeric-features=volume,quote_asset_volume,number